<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/oai_completion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="콜랩에서 열기"/></a>

저작권 (c) Microsoft Corporation. 모든 권리 보유.

MIT 라이선스에 따라 라이선스가 부여되었습니다.

# 자동 생성 기능을 사용하여 OpenAI 모델 조정하기

AutoGen은 대규모 언어 모델 튜닝을 위한 비용 효율적인 하이퍼파라미터 최적화 기법 [EcoOptiGen](https://arxiv.org/abs/2303.04673)을 제공합니다. 연구 결과에 따르면 하이퍼파라미터를 튜닝하면 LLM의 활용도를 크게 향상시킬 수 있습니다.
이 기능에 대한 설명서는 [여기](/docs/Use-Cases/AutoGen#enhanced-inference)에서 확인할 수 있습니다.

이 노트북에서는 코드 생성을 위해 OpenAI 모델을 튜닝합니다. 문서 문자열에서 프로그램을 합성하기 위해 OpenAI에서 공개한 [HumanEval 벤치마크](https://huggingface.co/datasets/openai_humaneval)를 사용합니다.

## 요구 사항

자동 생성에는 `Python>=3.8`이 필요합니다. 이 노트북 예제를 실행하려면 [blendsearch] 옵션으로 설치해 주세요:
```bash
pip install pyautogen[blendsearch]
```

In [1]:
# %pip install "pyautogen[blendsearch]~=0.1.0" datasets

## API 엔드포인트 설정

 이 함수는 API 키와 API 베이스가 해당 환경 변수 또는 로컬 txt 파일에 저장되어 있다고 가정합니다:
  - OpenAI API 키: os.environ["OPENAI_API_KEY"] 또는 `openai_api_key_file="key_openai.txt"`.
  - Azure OpenAI API 키: os.environ["AZURE_OPENAI_API_KEY"] 또는 `aoai_api_key_file="key_aoai.txt"`. 한 줄에 하나씩 여러 개의 키를 저장할 수 있습니다.
  - Azure OpenAI API 베이스: os.environ["AZURE_OPENAI_API_BASE"] 또는 `aoai_api_base_file="base_aoai.txt"`. 한 줄에 하나씩 여러 개의 베이스를 저장할 수 있습니다.
* `config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) 함수는 환경 변수 또는 json 파일에서 구성 목록을 로드합니다. 이 함수는 먼저 유효한 json 문자열이어야 하는 환경 변수 `env_or_file`을 찾습니다. 해당 변수를 찾을 수 없으면 같은 이름의 json 파일을 찾습니다. 그런 다음 filter_dict로 구성을 필터링합니다.

OpenAI API 키만 있거나 Azure OpenAI API 키 + 베이스만 있어도 괜찮습니다. 콜랩에서 이 노트북을 열면 왼쪽 패널의 파일 아이콘을 클릭한 다음 "파일 업로드" 아이콘을 선택해 파일을 업로드할 수 있습니다.


In [2]:
import autogen

endpoint_list = autogen.config_list_openai_aoai()
# the endpoint_list looks like this:
# endpoint_list = [
#     {
#         'api_key': '<your OpenAI API key here>',
#     },  # OpenAI API endpoint for gpt-4
#     {
#         'api_key': '<your first Azure OpenAI API key here>',
#         'api_base': '<your first Azure OpenAI API base here>',
#         'api_type': 'azure',
#         'api_version': '2023-03-15-preview',
#     },  # Azure OpenAI API endpoint for gpt-4
#     {
#         'api_key': '<your second Azure OpenAI API key here>',
#         'api_base': '<your second Azure OpenAI API base here>',
#         'api_type': 'azure',
#         'api_version': '2023-03-15-preview',
#     },  # another Azure OpenAI API endpoint for gpt-4
# ]

config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo-0301",
            "chatgpt-35-turbo-0301",
            "gpt-35-turbo-v0301",
            "gpt",
        },
    },
)
# the config_list looks like this:
# config_list = [
#     {
#         'model': 'gpt-3.5-turbo',
#         'api_key': '<your OpenAI API key here>',
#     },  # OpenAI API endpoint for gpt-3.5-turbo
#     {
#         'model': 'gpt-3.5-turbo',
#         'api_key': '<your first Azure OpenAI API key here>',
#         'api_base': '<your first Azure OpenAI API base here>',
#         'api_type': 'azure',
#         'api_version': '2023-06-01-preview',
#     },  # Azure OpenAI API endpoint for gpt-3.5-turbo
#     {
#         'model': 'gpt-35-turbo-v0301',
#         'api_key': '<your second Azure OpenAI API key here>',
#         'api_base': '<your second Azure OpenAI API base here>',
#         'api_type': 'azure',
#         'api_version': '2023-06-01-preview',
#     },  # another Azure OpenAI API endpoint for gpt-3.5-turbo with deployment name gpt-35-turbo-v0301
# ]


위에 제공된 두 가지 유틸리티 함수를 사용하지 않는 경우 원하는 다른 방법으로 목록을 정의할 수 있습니다.

## 데이터 세트 로드

먼저 휴먼밸 데이터셋을 로드합니다. 데이터 세트에는 164개의 예가 포함되어 있습니다. 처음 20개는 생성 하이퍼파라미터를 조정하는 데 사용하고 나머지는 평가에 사용합니다. 각 예제에서 "prompt"는 코드 생성을 유도하기 위한 프롬프트 문자열("정의"로 이름 변경)이고, "test"는 예제에 대한 단위 테스트를 위한 Python 코드이며, "entry_point"는 테스트할 함수 이름입니다.

In [3]:
import datasets

seed = 41
data = datasets.load_dataset("openai_humaneval")["test"].shuffle(seed=seed)
n_tune_data = 20
tune_data = [
    {
        "definition": data[x]["prompt"],
        "test": data[x]["test"],
        "entry_point": data[x]["entry_point"],
    }
    for x in range(n_tune_data)
]
test_data = [
    {
        "definition": data[x]["prompt"],
        "test": data[x]["test"],
        "entry_point": data[x]["entry_point"],
    }
    for x in range(n_tune_data, len(data))
]


Found cached dataset openai_humaneval (/home/vscode/.cache/huggingface/datasets/openai_humaneval/openai_humaneval/1.0.0/2955cebd73602e828fa8c0a424c594e5fab4ec863b316ca98f3d8fdb6a626e75)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /home/vscode/.cache/huggingface/datasets/openai_humaneval/openai_humaneval/1.0.0/2955cebd73602e828fa8c0a424c594e5fab4ec863b316ca98f3d8fdb6a626e75/cache-1e8448101c1b32e8.arrow


튜닝 예시를 확인하세요:

In [4]:
print(tune_data[1]["definition"])


def compare(game,guess):
    """I think we all remember that feeling when the result of some long-awaited
    event is finally known. The feelings and thoughts you have at that moment are
    definitely worth noting down and comparing.
    Your task is to determine if a person correctly guessed the results of several matches.
    You are given two arrays of scores and guesses of equal length, where each index shows a match. 
    Return an array of the same length denoting how far off each guess was. If they have guessed correctly,
    the value is 0; if not, the value is the absolute difference between the guess and the score.
    
    
    example:

    compare([1,2,3,4,5,1],[1,2,3,4,2,-2]) -> [0,0,0,0,3,3]
    compare([0,5,0,0,0,4],[4,1,1,0,0,-2]) -> [4,4,1,0,0,6]
    """



다음은 생성된 코드의 정확성을 검증하기 위한 단위 테스트 코드의 한 예입니다:

In [5]:
print(tune_data[1]["test"])

def check(candidate):

    # Check some simple cases
    assert candidate([1,2,3,4,5,1],[1,2,3,4,2,-2])==[0,0,0,0,3,3], "This prints if this assert fails 1 (good for debugging!)"
    assert candidate([0,0,0,0,0,0],[0,0,0,0,0,0])==[0,0,0,0,0,0], "This prints if this assert fails 1 (good for debugging!)"
    assert candidate([1,2,3],[-1,-2,-3])==[2,4,6], "This prints if this assert fails 1 (good for debugging!)"
    assert candidate([1,2,3,5],[-1,2,3,4])==[2,0,0,1], "This prints if this assert fails 1 (good for debugging!)"

    # Check some edge cases that are easy to work out by hand.
    assert True, "This prints if this assert fails 2 (also good for debugging!)"




## 성공 지표 정의

튜닝을 시작하기 전에 최적화하려는 성공 지표를 정의해야 합니다. 각 코드 생성 작업에 대해 모델을 사용하여 여러 후보를 생성한 다음 그중 하나를 선택할 수 있습니다. 최종적으로 선택된 응답이 단위 테스트를 통과할 수 있으면 해당 작업이 성공적으로 해결된 것으로 간주합니다. 그런 다음 작업 모음의 평균 성공률을 정의할 수 있습니다.

In [6]:
from functools import partial

eval_with_generated_assertions = partial(
    autogen.code_utils.eval_function_completions,
    assertions=partial(autogen.code_utils.generate_assertions, config_list=config_list),
    use_docker=False,
    # Please set use_docker=True if docker is available to run the generated code.
    # Using docker is safer than running the generated code directly.
)


이 함수는 먼저 각 문제에 대한 어설션 문을 생성합니다. 그런 다음 어설션을 사용하여 생성된 응답을 선택합니다.

## 튜닝 데이터를 사용하여 올바른 구성 찾기

AutoGen은 OpenAI 모델의 하이퍼파라미터 최적화를 위한 API를 제공합니다: autogen.Completion.tune`을 호출하고 튜닝된 구성으로 요청할 수 있습니다: autogen.Completion.create`를 제공합니다.

(로컬) 재현성과 비용 효율성을 위해 제어 가능한 시드를 사용하여 OpenAI의 응답을 캐시합니다.

In [7]:
autogen.Completion.set_cache(seed)

이렇게 하면 ".cache/{seed}"에 디스크 캐시가 생성됩니다. 캐시_경로_루트`를 `set_cache()`에서 ".cache"에서 다른 경로로 변경할 수 있습니다. 다른 시드에 대한 캐시는 별도로 저장됩니다.

### 튜닝 수행

튜닝은 최적화 예산에 따라 완료하는 데 시간이 걸릴 수 있습니다. 튜닝은 지정된 최적화 예산 내에서 수행됩니다.

* '추론_예산'은 벤치마크에서 인스턴스당 목표 평균 추론 예산입니다. 예를 들어 0.02는 목표 추론 예산이 0.02달러라는 의미이며, 텍스트 다빈치 모델을 사용하는 경우 1000토큰(입력 + 출력 합산)으로 해석됩니다.
* 'optimization_budget'은 튜닝을 수행하는 데 허용되는 총 예산입니다. 예를 들어 5는 총 5달러가 허용됨을 의미하며, 텍스트 다빈치 모델의 경우 25만 토큰이 허용됩니다.
* num_sumples`는 시도할 수 있는 다양한 하이퍼파라미터 구성의 개수입니다. 튜닝은 num_samples를 시도한 후 또는 최적화_예산이 소비된 후 둘 중 먼저 발생하는 시점 이후에 중지됩니다. -1은 시도 횟수에 제한이 없으며 실제 횟수는 `optimization_budget`에 의해 결정됨을 의미합니다.

사용자는 튜닝 데이터, 최적화 지표, 최적화 모드, 평가 기능, 검색 공간 등을 지정할 수 있습니다. 기본 검색 공간은

```python
default_search_space = {
    "model": tune.choice([[
        "text-ada-001",
        "text-babbage-001",
        "text-davinci-003",
        "gpt-3.5-turbo",
        "gpt-4",
    ]),
    "temperature_or_top_p": tune.choice(
        [
            {"temperature": tune.uniform(0, 1)},
            {"top_p": tune.uniform(0, 1)},
        ]
    ),
    "max_tokens": tune.lograndint(50, 1000),
    "n": tune.randint(1, 100),
    "prompt": "{프롬프트}",
}
```

사용자의 입력이 기본 검색 공간을 재정의할 수 있습니다.
예를 들어, 다음 코드는 프롬프트에 대한 세 가지 선택 사항과 중지 시퀀스에 대한 두 가지 선택 사항을 지정합니다. 기본 검색 공간은 사용자 입력에 나타나지 않는 하이퍼파라미터에 사용됩니다. gpt-4에 액세스할 수 없거나 모델 선택을 수정하려는 경우 모델에 대해 다른 검색 공간을 제공할 수 있습니다.

In [8]:
config, analysis = autogen.Completion.tune(
    data=tune_data,  # the data for tuning
    metric="success",  # the metric to optimize
    mode="max",  # the optimization mode
    eval_func=eval_with_generated_assertions,  # the evaluation function to return the success metrics
    # log_file_name="logs/humaneval.log",  # the log file name
    inference_budget=0.05,  # the inference budget (dollar per instance)
    optimization_budget=1,  # the optimization budget (dollar in total)
    # num_samples can further limit the number of trials for different hyperparameter configurations;
    # -1 means decided by the optimization budget only
    num_samples=-1,
    prompt=[
        "{definition}",
        "# Python 3{definition}",
        "Complete the following Python function:{definition}",
    ],  # the prompt templates to choose from
    stop=[["\nclass", "\ndef", "\nif", "\nprint"], None],  # the stop sequences
    config_list=endpoint_list,  # optional: a list of endpoints to use
    allow_format_str_template=True,  # whether to allow format string template
)


[I 2023-07-30 04:19:08,150] A new study created in memory with name: optuna
[I 2023-07-30 04:19:08,153] A new study created in memory with name: optuna


[flaml.tune.tune: 07-30 04:19:08] {805} INFO - trial 1 config: {'prompt': 1, 'stop': 0, 'subspace': {'model': 'text-ada-001', 'max_tokens': 148, 'temperature_or_top_p': {'top_p': 0.755486898036596}, 'n': 27}}
[flaml.tune.tune: 07-30 04:22:35] {197} INFO - result: {'index_selected': 26.0, 'succeed_assertions': 0.0, 'success': 0.0, 'gen_cost': 0.000460625, 'assertions': 'assert vowels_count("abcde") == 2\nassert vowels_count("ACEDY") == 3', 'total_cost': 0.010514800000000003, 'cost': 0.010514800000000003, 'inference_cost': 0.00023534000000000003, 'training_iteration': 0, 'config': {'prompt': 1, 'stop': 0, 'subspace': {'model': 'text-ada-001', 'max_tokens': 148, 'temperature_or_top_p': {'top_p': 0.755486898036596}, 'n': 27}}, 'config/prompt': 1, 'config/stop': 0, 'config/subspace': {'model': 'text-ada-001', 'max_tokens': 148, 'temperature_or_top_p': {'top_p': 0.755486898036596}, 'n': 27}, 'experiment_tag': 'exp', 'time_total_s': 207.29033374786377}
[flaml.tune.tune: 07-30 04:22:35] {805} 

출력 튜닝 결과 ###

튜닝이 끝나면 설정과 오토젠이 찾은 결과를 인쇄할 수 있습니다:

In [9]:
print("optimized config", config)
print("best result on tuning data", analysis.best_result)

optimized config {'prompt': '# Python 3{definition}', 'stop': ['\nclass', '\ndef', '\nif', '\nprint'], 'model': 'text-davinci-003', 'max_tokens': 148, 'n': 27, 'top_p': 0.755486898036596}
best result on tuning data {'index_selected': 2.35, 'succeed_assertions': 0.95, 'success': 0.65, 'gen_cost': 0.000460625, 'assertions': 'assert vowels_count("abcde") == 2\nassert vowels_count("ACEDY") == 3', 'total_cost': 0.8658043000000002, 'cost': 0.8357800000000002, 'inference_cost': 0.04093000000000001, 'training_iteration': 0, 'config': {'prompt': 1, 'stop': 0, 'subspace': {'model': 'text-davinci-003', 'max_tokens': 148, 'temperature_or_top_p': {'top_p': 0.755486898036596}, 'n': 27}}, 'config/prompt': 1, 'config/stop': 0, 'config/subspace': {'model': 'text-davinci-003', 'max_tokens': 148, 'temperature_or_top_p': {'top_p': 0.755486898036596}, 'n': 27}, 'experiment_tag': 'exp', 'time_total_s': 62.81497287750244}


튜닝된 구성으로 요청하기 ###

예제 작업에 대한 요청에 조정된 구성을 적용할 수 있습니다:

In [10]:
response = autogen.Completion.create(context=tune_data[1], config_list=endpoint_list, **config)
print(response)
print(eval_with_generated_assertions(autogen.Completion.extract_text(response), **tune_data[1]))


{
  "id": "cmpl-7hsFhPX6faeWYaT4y3C7IkQAgNbZR",
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "object": "text_completion",
  "created": 1690691005,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "    results = []\n    for i in range(len(game)):\n        if game[i] == guess[i]:\n            results.append(0)\n        else:\n            results.append(abs(game[i]-guess[i]))\n    return results",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    },
    {
      "text": "    result = []\n    for i in range(len(game)):\n        result.append(abs(game[i] - guess[i]))\n    return result",
      "index": 1,
      "logprobs": null,
      "finish_reason": "stop"
    },
    {
      "text": "    result = []\n    for i in range(len(game)):\n        result.append(abs(game[i]-guess[i]))\n    return result",
      "index": 2,
      "l

테스트 데이터의 성공률 평가 ###

autogen.Completion.test`를 사용하여 튜닝된 구성으로 전체 데이터 세트의 성능을 평가할 수 있습니다. 다음 코드는 144개의 테스트 데이터 인스턴스를 모두 평가하는 데 시간이 걸립니다. 코멘트를 제거하고 실행하면 비용은 약 $6입니다.

In [12]:
# result = autogen.Completion.test(test_data, config_list=endpoint_list, **config)
# print("performance on test data with the tuned config:", result)

performance on test data with the tuned config: {'index_selected': 5.222222222222222, 'succeed_assertions': 0.8402777777777778, 'success': 0.7569444444444444, 'gen_cost': 0.00044632638888888885, 'cost': 5.704979999999999, 'inference_cost': 0.03961791666666666}


결과는 추론 예산과 최적화 예산에 따라 달라집니다.
